In [1]:
import requests
import json
import os
import pandas as pd
import numpy as np

In [202]:
te = pd.read_csv('/home/julian/Cursos/Ironhack/Proyectos/Proyecto2/temperature.csv')

In [135]:
te.columns

Index(['datetime', 'Vancouver', 'Portland', 'San Francisco', 'Seattle',
       'Los Angeles', 'San Diego', 'Las Vegas', 'Phoenix', 'Albuquerque',
       'Denver', 'San Antonio', 'Dallas', 'Houston', 'Kansas City',
       'Minneapolis', 'Saint Louis', 'Chicago', 'Nashville', 'Indianapolis',
       'Atlanta', 'Detroit', 'Jacksonville', 'Charlotte', 'Miami',
       'Pittsburgh', 'Toronto', 'Philadelphia', 'New York', 'Montreal',
       'Boston', 'Beersheba', 'Tel Aviv District', 'Eilat', 'Haifa',
       'Nahariyya', 'Jerusalem'],
      dtype='object')

In [136]:
te.shape

(45253, 37)

In [137]:
nulls = [te[i].isna().value_counts() for i in te.columns]

In [138]:
print(nulls[2])

False    45252
True         1
Name: Portland, dtype: int64


### eliminamos columnas inecesarias (solo usa y canada)

In [203]:
def keep_columns(x, y):
    """
    Toma como argumento la lista de las columnas que quiero conservar y el nombre del data frame.
    Recorre el indice de columnas del data frame correspondiente y elimina las
    columnas que no quiero conservar
    """
    relevant = x
    for i in y.columns:
        if i not in relevant:
            y.drop(columns=[i], inplace=True)

In [204]:
keep_columns(['datetime', 'Vancouver', 'Portland', 'San Francisco', 'Seattle',
       'Los Angeles', 'San Diego', 'Las Vegas', 'Phoenix', 'Albuquerque',
       'Denver', 'San Antonio', 'Dallas', 'Houston', 'Kansas City',
       'Minneapolis', 'Saint Louis', 'Chicago', 'Nashville', 'Indianapolis',
       'Atlanta', 'Detroit', 'Jacksonville', 'Charlotte', 'Miami',
       'Pittsburgh', 'Toronto', 'Philadelphia', 'New York', 'Montreal',
       'Boston'], te)

### Agrupamos por fecha y mediana

In [141]:
def mediana_mensual(x):
    x['Date'] = x.datetime.apply(lambda y: y[:7])
    return x.groupby("Date").median()

In [142]:
tem = mediana_mensual(te)

### transformamos a celsius

In [37]:
def celsius_conv(row):
    for i in tem.index:
           row[i] = row[i] - 273.15
    return row
          

temc = tem.apply(lambda row : celsius_conv(row))

In [101]:
temc.head(3)

,Vancouver,Portland,San Francisco,Seattle,Los Angeles,San Diego,Las Vegas,Phoenix,Albuquerque,Denver,...,Detroit,Jacksonville,Charlotte,Miami,Pittsburgh,Toronto,Philadelphia,New York,Montreal,Boston
Date,,,,,,,,,,,,,,,,,,,,,
2012-10,10.050,11.630,15.730,10.630,19.260,18.85,20.35,23.740,13.83,9.730,...,11.370,23.24,15.505086,26.25,11.2400,10.11,14.190,14.750,10.600,13.150
2012-11,7.165,8.405,13.510,7.900,15.500,15.52,14.32,17.705,7.45,7.190,...,4.010,14.86,8.470000,21.66,3.4900,3.55,5.420,5.695,1.255,4.335
2012-12,4.025,5.110,10.665,5.065,13.245,13.13,8.82,11.405,1.31,0.275,...,2.455,15.96,9.195000,22.19,3.0025,0.75,4.575,4.490,-3.135,1.935


### Repetimos proceso con datos de humedad

In [40]:
h = pd.read_csv('/home/julian/Cursos/Ironhack/Proyectos/Proyecto2/humidity.csv')

In [42]:
keep_columns(['datetime', 'Vancouver', 'Portland', 'San Francisco', 'Seattle',
       'Los Angeles', 'San Diego', 'Las Vegas', 'Phoenix', 'Albuquerque',
       'Denver', 'San Antonio', 'Dallas', 'Houston', 'Kansas City',
       'Minneapolis', 'Saint Louis', 'Chicago', 'Nashville', 'Indianapolis',
       'Atlanta', 'Detroit', 'Jacksonville', 'Charlotte', 'Miami',
       'Pittsburgh', 'Toronto', 'Philadelphia', 'New York', 'Montreal',
       'Boston'], h)

In [100]:
def mediana_mensual(x):
    x['Date'] = x.datetime.apply(lambda y: y[:7])
    return x.groupby("Date").median()

In [45]:
hm = mediana_mensual(h)

In [ ]:
tecm['Carbon Levels'] = 

In [ ]:
for i in range(365):
    c.drop([i], axis=0, inplace=True)

### Obtengo los datos de la API

In [10]:
def get_carbon_data(url):
    url = f"{url}"
    headers = {
    'x-rapidapi-host': "daily-atmosphere-carbon-dioxide-concentration.p.rapidapi.com",
    'x-rapidapi-key': "6f91c81d8fmshc83b86b1b6c6c5cp108956jsnef221ca08c9a"
    }
    response = requests.request("GET", url, headers=headers)
    t = response.json()
    return t

In [233]:
c = pd.DataFrame(t['co2'])

In [230]:
type(c.cycle.unique()[0])

str

#### Creamos cd con todos los años para analizar dsp

In [60]:
def cod_agrupado(x):
    return x.groupby(["year", "month"]).agg({"cycle": "median"})

In [61]:
cd = cod_agrupado(c)

In [144]:
cd.head()

cycle
year month        
2011 1      391.68
     10     389.47
     11     391.38
     12     392.72
     2      391.73

#### Escalamos a cd para que tenga solo los años que nos interesan

In [234]:
dates = pd.period_range(start='2011-01-01', end='2021-08-27', freq='D')

In [235]:
c['datetime'] = [str(dates[i]) for i in range(len(dates))]

In [236]:
c.head(3)

,year,month,day,cycle,trend,datetime
0,2011,1,1,391.26,389.73,2011-01-01
1,2011,1,2,391.29,389.73,2011-01-02
2,2011,1,3,391.33,389.74,2011-01-03


In [237]:
keep_columns(['datetime', 'cycle', 'trend'], c)

In [238]:
c.head()

,cycle,trend,datetime
0,391.26,389.73,2011-01-01
1,391.29,389.73,2011-01-02
2,391.33,389.74,2011-01-03
3,391.36,389.75,2011-01-04
4,391.39,389.75,2011-01-05


In [239]:
c['Date'] = c.datetime.apply(lambda y: y[:7])

In [245]:
c.groupby("Date").median()

""
Date
2011-01
2011-02
2011-03
2011-04
2011-05
...
2021-04
2021-05
2021-06


## INTENTAR METIENDOLO EN EL FRAM DE TEMPS Y DESP GROUPBY